# Testing tools

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

def fetch_portfolio_data(tickers: list, days=30):
    """Downloads portfolio data"""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)

    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date.strftime('%Y-%m-%d'),
        end=end_date.strftime('%Y-%m-%d'),
        group_by='ticker'
    )
    return {ticker: data[ticker] for ticker in tickers}

portfolio = ["AAPL", "MSFT", "TSLA"]
stock_data = fetch_portfolio_data(portfolio)
stock_data['AAPL'][:10]

/tmp/ipython-input-2-2656228369.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  3 of 3 completed


Price,Open,High,Low,Close,Volume
Date,,,,,
2025-05-28,200.589996,202.729996,199.899994,200.419998,45339700
2025-05-29,203.580002,203.809998,198.509995,199.949997,51396800
2025-05-30,199.369995,201.960007,196.779999,200.850006,70819900
2025-06-02,200.279999,202.130005,200.119995,201.699997,35423300
2025-06-03,201.350006,203.770004,200.960007,203.270004,46381600
2025-06-04,202.910004,206.240005,202.100006,202.820007,43604000
2025-06-05,203.500000,204.750000,200.149994,200.630005,55126100
2025-06-06,203.000000,205.699997,202.050003,203.919998,46607700
2025-06-09,204.389999,206.000000,200.020004,201.449997,72862600


In [ ]:
import pandas as pd

def analyze_trends(data: dict):
    """pandas trends analyse"""
    reports = {}
    for ticker, df in data.items():
        last_close = df['Close'][-1]
        sma_10 = df['Close'].rolling(10).mean()[-1]

        reports[ticker] = {
            "last_price": last_close.item(),
            "trend": "up ↑" if last_close > sma_10 else "down ↓",
            "volatility": df['Close'].pct_change().std()
        }
    return reports

analysis = analyze_trends(stock_data)
analysis

/tmp/ipython-input-3-108356434.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_close = df['Close'][-1]
/tmp/ipython-input-3-108356434.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sma_10 = df['Close'].rolling(10).mean()[-1]


{'AAPL': {'last_price': 201.0,
  'trend': 'up ↑',
  'volatility': 0.010650301843965615},
 'MSFT': {'last_price': 497.45001220703125,
  'trend': 'up ↑',
  'volatility': 0.006264411328982318},
 'TSLA': {'last_price': 325.7799987792969,
  'trend': 'down ↓',
  'volatility': 0.046404621158714035}}

In [ ]:
import feedparser

def get_financial_news(ticker: str, limit=5):
    """Parsing of RSS-feed Yahoo Finance"""
    url = f"https://feeds.finance.yahoo.com/rss/2.0/headline?s={ticker}&region=US&lang=en-US"
    news = feedparser.parse(url)
    # for item in news:
    #     print(item, news[item])
    return [
        {"title": entry.title, "timestompt": entry.published, "summary": entry.summary}
        for entry in news.entries[:limit]
    ]

aapl_news = get_financial_news("AAPL")
aapl_news

[{'title': 'Apple revamps EU App Store terms to avert more fines',
  'timestompt': 'Fri, 27 Jun 2025 10:44:04 +0000',
  'summary': "Apple has revamped its app store policies in the European Union with hopes of fending off escalating fines under the 27-nation bloc's digital competition regulations.  The bloc's executive Commission punished Apple for preventing app makers from pointing users to cheaper options outside its App Store, and gave it a 60-day deadline, which expired Thursday, to avoid additional, periodic fines.  The changes made by Apple will make it easier for app makers to point users to better deals on digital products and options to pay for them outside of Apple's own App Store, including other websites, apps or alternative app stores."},
 {'title': 'Meet the Only "Magnificent Seven" Stock That Is Cheaper Than the S&P 500 (According to This Key Metric)',
  'timestompt': 'Fri, 27 Jun 2025 10:17:00 +0000',
  'summary': 'It has a diversified business but is still heavily dep

In [ ]:
from transformers import pipeline

sentiment_analyzer = pipeline(
    "text-classification",
    model="yiyanghkust/finbert-tone",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

Device set to use cuda


In [ ]:
result = sentiment_analyzer(aapl_news[4]["summary"])
label = result[0]['label']
label

'Neutral'

In [ ]:
for item in aapl_news:
    # print(item["summary tone"])
    item["summary tone"] = sentiment_analyzer(item["summary"])[0]["label"]

In [ ]:
# !pip install bs4 #fake_useragent

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent


def get_yahoo_article_text(url):
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Ошибка доступа к странице: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Заголовок статьи
        title_tag = soup.find('h1')
        title = title_tag.get_text(strip=True) if title_tag else "Без заголовка"

        # Контент статьи
        paragraphs = soup.find_all('div', class_=lambda x:
                                   x and 'article yf-l7apfj' in x or
                                   x and 'max-w-full' in x)
        article_text = '\n\n'.join(p.get_text(strip=True) for p in paragraphs)

        if not article_text:
            print("Текст статьи не найден.")
            return None

        return {
            'title': title,
            'text': article_text
        }

    except Exception as e:
        print(f"Ошибка при извлечении статьи: {e}")
        return None

In [32]:
# url = "https://finance.yahoo.com/news/investing-10k-magnificent-7-stocks-180100695.html?.tsrc=rss"
# url = 'https://www.fool.com/investing/2025/06/29/top-buffett-stocks-buy-and-hold-long-haul/?.tsrc=rss'
# result = get_yahoo_article_text(url)

# if result:
#     print("Заголовок:", result['title'])
#     print("\nТекст статьи:\n", result['text'])

# Agent

In [ ]:
from huggingface_hub import login

login("PLACE_YOUR_TOKEN")

In [2]:
from smolagents import ToolCallingAgent, TransformersModel, tool
from typing import List, Dict, Type
import torch
import yfinance as yf
import feedparser
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [3]:
# ticker="AAPL"
# url = f"https://feeds.finance.yahoo.com/rss/2.0/headline?s={ticker}&region=US&lang=en-US"

# feedparser.parse(url).entries[:2]

In [29]:
@tool
def get_ticker_information(ticker: str) -> str:
    """
    This is a tool that returns the most information about the ticker

    Args:
        ticker: The ticker for which to get info.
    """
    return yf.Ticker(ticker).info

@tool
def get_stock_price(ticker: str) -> str:
    """
    Returns stock current price
    Args:
        ticker: The ticker for which to get info.
    """
    return str(yf.Ticker(ticker).info['currentPrice'])

@tool
def get_financial_news_links(ticker: str, limit: int =5) -> List[Dict[str,str]]:
    """
    Parsing of RSS-feed Yahoo Finance.

    Args:
        ticker: The ticker for which to get info.
        limit: Sets the maximum of getting news number
    """
    url = f"https://feeds.finance.yahoo.com/rss/2.0/headline?s={ticker}&region=US&lang=en-US"
    try:
        news = feedparser.parse(url)
        return [entry.link for entry in news.entries[:limit]]
    except requests.exceptions.RequestException as e:
        return f"Error parisng yahoo finance: {str(e)}"

@tool
def get_article_link(links: str, i: int) -> str:
    """
    Extracts link from links list

    Args:
        links: links to the articles.
        i: which item to pick
    """
    return links.split(",")[i]

@tool
def get_article_text(url: str) -> str:
    """
    Extracts text from url link of an article.

    Args:
        url: url link to the article.
    """
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Ошибка доступа к странице: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # article title
        title_tag = soup.find('h1')
        title = title_tag.get_text(strip=True) if title_tag else "Без заголовка"

        # article content
        paragraphs = soup.find_all('div', class_=lambda x:
                                   x and 'article yf-l7apfj' in x or
                                   x and 'max-w-full' in x)
        article_text = '\n\n'.join(p.get_text(strip=True) for p in paragraphs)

        if not article_text:
            print("Text of the article not found")
            print(f"{title=}")
            return None

        return {
            'title': title,
            'text': article_text
        }

    except Exception as e:
        print(f"Error: {e}")
        return None

In [21]:
url = get_financial_news("AAPL", limit=1)
get_article_text(url[0]["link"])


{'title': '2 Top Buffett Stocks to Buy and Hold for the Long Haul',
 'text': 'Our Purpose:To make the world smarter, happier, and richer.Founded in 1993, The Motley Fool is a financial services company dedicated to making the world smarter, happier, and richer. The Motley Fool reaches millions of people every month through our premium investing solutions, free guidance and market analysis on Fool.com, top-rated podcasts, and non-profit The Motley Fool Foundation.\n\nOur Purpose:To make the world smarter, happier, and richer.Founded in 1993, The Motley Fool is a financial services company dedicated to making the world smarter, happier, and richer. The Motley Fool reaches millions of people every month through our premium investing solutions, free guidance and market analysis on Fool.com, personal finance education, top-rated podcasts, and non-profit The Motley Fool Foundation.\n\nOur Purpose:To make the world smarter, happier, and richer.Founded in 1993, The Motley Fool is a financial s

In [6]:
# model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct" <- too big, mess with instructions
# model_name = "HuggingFaceTB/SmolLM2-360M-Instruct" <- too small, do not recognize instruments
# model_name ="google/flan-t5-small"                 <- doesn't fit to TransformersModel class

model_name ="Qwen/Qwen3-0.6B"                      # <- optimal with size + thinking
drive_llm = TransformersModel(model_id=model_name,
                              device_map="auto",
                              # max_new_tokens=1000
                              )

`max_new_tokens` not provided, using this default value for `max_new_tokens`: 4096


In [30]:
tools = [
    get_ticker_information,
    get_stock_price,
    get_financial_news_links,
    get_article_link,
    get_article_text
]
tools_names = [tool.name for tool in tools]
agent = ToolCallingAgent(tools=tools,
                         model=drive_llm,
                         verbosity_level=2)

In [31]:
# print("ToolCallingAgent:", agent.run("What is the current price of Apple?"))
json_format = 'Action: {"name": "<instrument_name>", "arguments": {<Key>: <value>}}'
system_prompt = f'''
You are working as a finantial agent and have to use only from listed instruments:\n{tools_names}.
You have to return only in the following JSON format: {json_format}. Do not add any extra text before and after.'''
with torch.no_grad():
    print("ToolCallingAgent:", agent.run(f"{system_prompt}\n\nAnswer: What is the current news of Apple stock? Is it positive or negative?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are working as a finantial agent and have to use only from listed instruments:                              │
│ ['get_ticker_information', 'get_stock_price', 'get_financial_news_links', 'get_article_link',                   │
│ 'get_article_text'\].                                                                                           │
│ You have to return only in the following JSON format: Action: {"name": "<instrument_name>", "arguments":        │
│ {<Key>: <value>}}. Do not add any extra text before and after.                                                  │
│                                                                                                                 │
│ Answer: What is the current news of Apple stock? Is it positive or negative?                                    │
│                                                                                                                 │
╰─ TransformersModel - Qwen/Qwen3-0.6B ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
Okay, the user is asking for the current news of Apple stock and whether it's positive or negative. Let me check   
the available tools. The tools provided are get_ticker_information, get_stock_price, get_financial_news_links,     
get_article_link, and get_article_text.                                                                            
                                                                                                                   
The user wants to know the news about Apple's stock. The financial_news_links tool can parse news from Yahoo       
Finance. So I should use that. The arguments needed are the ticker for Apple. The ticker is AAPL. Let me make sure 
to use the correct parameter. Then, after getting the news links, I can extract the text and determine if it's     
positive or negative. But since the user is asking for both the news and the sentiment, maybe I need to call the   
financial_news_links twice: once to get the links and once to get the text. However, the user's question is a      
single query. So perhaps the best approach is to first call get_financial_news_links with the ticker AAPL to get   
the news links, then use get_article_text on those links to extract the text. But since the user is asking for the 
news and sentiment, maybe I need to check both. Wait, but the tools can only be used once.

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_financial_news_links' with arguments: {'ticker': 'AAPL'}                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |'https://www.fool.com/investing/2025/06/29/top-buffett-stocks-buy-and-hold-long-haul/?.tsrc=rss', 
'https://www.fool.com/investing/2025/06/29/5-artificial-intelligence-ai-stocks-are-worth-over/?.tsrc=rss', 
'https://www.fool.com/investing/2025/06/29/3-artificial-intelligence-ai-stocks-that-still-loo/?.tsrc=rss', 
'https://www.thestreet.com/investing/veteran-fund-manager-who-predicted-nvidia-stock-rally-makes-surprising-move?.t
src=rss', 'https://finance.yahoo.com/news/investing-10k-magnificent-7-stocks-180100695.html?.tsrc=rss']

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
Okay, let's see. The user is asking about the current news of Apple stock and whether it's positive or negative.   
The previous interactions involved using tools to get financial news, and the latest observation from the          
'get_article_text' tool provided a detailed article about Apple and Berkshire Hathaway.                            
                                                                                                                   
Looking at the observation, the article mentions that Apple has a "better business than any other we own           
[outright]." It also notes that despite a decline of about 18% in 2025, Apple's track record of innovation,        
shareholder returns, and support from Warren Buffett make it a compelling case for long-term investors. The stock  
trades at 31 times its trailing earnings, slightly above the median.                                               
                                                                                                                   
The article also talks about Apple's investment in AI, specifically Apple Intelligence, which CEO Tim Cook         
highlighted. This suggests positive sentiment. Additionally, the mention of a $100 billion share repurchase program
and a quarterly dividend of $0.26 per share adds to the positive outlook.                                          
                                                                                                                   
Since the article states that despite the decline, Apple is still a good investment and that the valuations are    
favorable, it's clear that the news is positive. Therefore, the answer should confirm[38;2;230;237;243;48

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "The current news of Apple stock is positive. Despite a │
│ decline of approximately 18% in 2025, Apple's track record of innovation, shareholder returns, and support from │
│ Warren Buffett make it a compelling long-term investment."}                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The current news of Apple stock is positive. Despite a decline of approximately 18% in 2025, Apple's 
track record of innovation, shareholder returns, and support from Warren Buffett make it a compelling long-term 
investment.

[Step 3: Duration 21.38 seconds| Input tokens: 6,400 | Output tokens: 2,176]

ToolCallingAgent: The current news of Apple stock is positive. Despite a decline of approximately 18% in 2025, Apple's track record of innovation, shareholder returns, and support from Warren Buffett make it a compelling long-term investment.
